# Simple RNN Tensorflow

In [33]:
# this cell is for imports
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import pandas as pd
import collections
import os
import time
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/shunlin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
# getting jokes from kaggle dataset
kaggle_jokes_raw = pd.read_csv('input/kaggle/shortjokes.csv')

In [35]:
# see the head of the jokes
kaggle_jokes_raw.head()

,ID,Joke
0,1,"[me narrating a documentary about narrators] ""..."
1,2,Telling my daughter garlic is good for you. Go...
2,3,I've been going through a really rough period ...
3,4,"If I could have dinner with anyone, dead or al..."
4,5,Two guys walk into a bar. The third guy ducks.


In [36]:
# the rest of this notebook follows code from https://www.tensorflow.org/tutorials/sequences/text_generation
# build a set of vocab (characters)
joke_lst = kaggle_jokes_raw["Joke"].tolist()
text = joke_lst[0]
# use the first 1000 jokes to play with model
for joke in joke_lst[1:5000]:
    text += '\n\n'
    text += joke
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

93 unique characters


In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [6]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  '+' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  ...
}


In [7]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'[me narrating' ---- characters mapped to int ---- > [60 78 70  1 79 66 83 83 66 85 74 79 72]


In [8]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

[
m
e
 
n


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'[me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTell'
'ing my daughter garlic is good for you. Good immune system and keeps pests away.Ticks, mosquitos, vam'
"pires... men.\n\nI've been going through a really rough period at work this week It's my own fault for "
'swapping my tampax for sand paper.\n\nIf I could have dinner with anyone, dead or alive... ...I would c'
"hoose alive. -B.J. Novak-\n\nTwo guys walk into a bar. The third guy ducks.\n\nWhy can't Barbie get pregn"


In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '[me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTel'
Target data: 'me narrating a documentary about narrators] "I can\'t hear what they\'re saying cuz I\'m talking"\n\nTell'


In [12]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 60 ('[')
  expected output: 78 ('m')
Step    1
  input: 78 ('m')
  expected output: 70 ('e')
Step    2
  input: 70 ('e')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 79 ('n')
Step    4
  input: 79 ('n')
  expected output: 66 ('a')


In [13]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences, 
# so it doesn't attempt to shuffle the entire sequence in memory. Instead, 
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<DatasetV1Adapter shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(
        tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
        rnn(rnn_units,
            return_sequences=True, 
            recurrent_initializer='glorot_uniform',
            stateful=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 93) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           23808     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 93)            95325     
Total params: 4,054,365
Trainable params: 4,054,365
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([53, 21,  8, 66, 33,  7, 48, 37, 66,  3, 13, 89,  9, 45, 59, 11, 56,
       87, 38, 73, 80, 45, 77, 19, 69, 90, 48, 13, 20,  5, 15, 26,  9,  6,
       60, 49, 76, 79, 33, 32, 49, 48, 24, 85,  9, 78, 17, 63, 71, 64, 30,
       71, 84, 15, 10, 48, 40, 54, 33, 22, 16, 40, 31, 59, 31, 69, 13, 35,
       65, 42, 68, 88, 17, 41, 72, 85,  1, 34, 92, 14, 34, 32, 59, 16, 13,
       31, 54, 24, 38, 43, 71, 87, 56, 66, 23, 72, 36, 34, 24, 29])

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'bating cattle? Beef Strokenoff\n\nDid you hear about the magic tractor? It went down a hill and turned'

Next Char Predictions: 
 'T4\'a@&ODa",x(LZ*WvEhoLl2dyO,3$.9(%[Pkn@?PO7t(m0^f_=fs.)OGU@5/G>Z>d,B`Icw0Hgt A~-A?Z/,>U7EJfvWa6gCA7<'


In [23]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") 
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 93)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.532006


In [24]:
model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss)

In [25]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
EPOCHS=3

In [27]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])


Epoch 1/3
74/74 [==============================] - 641s 9s/step - loss: 3.3882
Epoch 2/3
74/74 [==============================] - 595s 8s/step - loss: 2.4252
Epoch 3/3
74/74 [==============================] - 3041s 41s/step - loss: 2.2288


In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_3'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            23808     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3935232   
_________________________________________________________________
dense_1 (Dense)              (1, None, 93)             95325     
Total params: 4,054,365
Trainable params: 4,054,365
Non-trainable params: 0
_________________________________________________________________


In [31]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing) 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [32]:
print(generate_text(model, start_string=u"You are "))

W0307 23:46:21.941138 140736103486336 deprecation.py:323] From <ipython-input-31-d014360b5fe2>:28: multinomial (from tensorflow.python.ops.random_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.random.categorical` instead.


You are extoun? Her attring a lyour whome aloum avelre.

I has dind!

I joko a babe gut's of prouskestin- burtpe an llirs syou fough

Pophisp cauken't of is ined treanigh clatone cllade a brave axk peeling cemeded the torin justyouly fares

I ald,? Heg hake on lyick

I lakn you gromadermart bater at poobe rest endeed in orent ancupsw? (I my kid shand 1 Gon`lio the betit! The mrad ameat nowk houns he. Chous? 
OR As/Je3c.: [QARn Cone in a bat (/menow dow a bope becauser.. tom Bestared

I ly faren." *nowsay a anm hal you'n to jokmey leRiss. apsuted loured tompus

2. Aspunill cair blautich "Id matith in M6!UY just dpain theve. Meabie me you rave you car thenidif e purtar..!.

He: Grameny't jukn whouryt's tomer S> Pradis Dilly fir, af the grutt? Well tile ther E htowed fit? It's shaw do". Tut have soring ot thaw bo with! Wear ant sot ta bear on tha palllipall 2Y Jey.


Nit wos a bauve

Ways I One Se can's "heerd Aringnoas Soir, up asthend Wean'tes a sash is preapie, if cally shent,w, maste?